# 1 图像部分
- pysekiro
    - img_tools
        - \__init__.py
        - adjustment.py (游戏窗口校准)
        - get_status.py (状态获取)
        - get_vertices.py (顶点位置获取)
        - grab_screen.py (屏幕图像抓取)

## 1.1 屏幕图像抓取

[grab_screen.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/CN/pysekiro/img_tools/grab_screen.py)  
源代码： [Sentdex/pygta5/blob/master/grabscreen.py](https://github.com/Sentdex/pygta5/blob/master/grabscreen.py)

In [ ]:
# 运行以下代码，就会抓取当前的屏幕图像
import cv2
from pysekiro.img_tools.grab_screen import get_screen

screen = get_screen()
cv2.imshow('screen', screen)
cv2.waitKey(0)

print('形状', screen.shape)

## 1.2 顶点位置获取

[get_vertices.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/CN/pysekiro/img_tools/get_vertices.py)

![demo.gif](https://github.com/ricagj/pysekiro/blob/main/imgs/demo.gif?raw=true)

如上面的演示所示（demo.gif），在弹出来的窗口中用鼠标左键按顺序依次点击**左下**，**左上**，**右上**，**右下**，一共 4 次，然后按键盘上的“ESC”键，就会自动返回 x, x_w, y, y_h。  
（注意：这个点击的顺序是规定好的，点击的次数也是规定好的）

In [ ]:
import cv2

from pysekiro.img_tools.get_vertices import get_xywh
from pysekiro.img_tools.grab_screen import get_screen

img = get_screen()

get_xywh(img)

## 1.3 状态获取

[get_status.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/CN/pysekiro/img_tools/get_status.py)

![get_status](https://github.com/ricagj/pysekiro/blob/main/imgs/status.jpg?raw=true)

要获取的状态有：自身生命，自身架势，目标生命，目标架势。  
- 获取方法
    1. 从 [get_vertices.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/CN/pysekiro/img_tools/get_vertices.py) 获取 **x, x_w, y, y_h** 。
    2. 图像处理
    3. 量化成数字

### 1.3.1 生命值的获取

1. 提取感兴趣区域
2. 颜色通道分离，取G(Green)通道
3. 图像阈值处理
4. 转化为数值

~~~python
# 获取自身生命
def get_Self_HP(img):
    img_roi = roi(img, x=48, x_w=305, y=409, y_h=409+1)

    b, g ,r =cv2.split(img_roi)    # 颜色通道分离

    retval, img_th = cv2.threshold(g, 50, 255, cv2.THRESH_TOZERO)              # 图像阈值处理，像素点的值低于50的设置为0
    retval, img_th = cv2.threshold(img_th, 70, 255, cv2.THRESH_TOZERO_INV)    # 图像阈值处理，像素点的值高于70的设置为0

    target_img = img_th[0]
    if 0 in target_img:
        Self_HP = np.argmin(target_img)
    else:
        Self_HP = len(target_img)

    return Self_HP

# 获取目标生命
def get_Target_HP(img):
    img_roi = roi(img, x=48, x_w=216, y=41, y_h=41+1)

    b, g ,r =cv2.split(img_roi)    # 颜色通道分离

    retval, img_th = cv2.threshold(g, 25, 255, cv2.THRESH_TOZERO)             # 图像阈值处理，像素点的值低于25的设置为0
    retval, img_th = cv2.threshold(img_th, 70, 255, cv2.THRESH_TOZERO_INV)    # 图像阈值处理，像素点的值高于70的设置为0

    target_img = img_th[0]
    if 0 in target_img:
        Target_HP = np.argmin(target_img)
    else:
        Target_HP = len(target_img)
    
    return Target_HP
~~~

![demo_get_Self_HP.gif](https://github.com/ricagj/pysekiro/blob/main/imgs/demo_get_Self_HP.gif?raw=true)

### 1.3.2 架势的获取

- 只读取架势的右半部分
1. 提取感兴趣区域
2. 颜色通道分离，取R(Red)通道
3. 等待架势中线（白线）出现
    - 用 cv2.Canny() 检测边缘
4. 转化为数值

~~~python
# 获取自身架势
def get_Self_Posture(img):
    img_roi = roi(img, x=401, x_w=490, y=389, y_h=389+1)
    b, g ,r =cv2.split(img_roi)    # 颜色通道分离

    white_line = r[0][0]
    if 155 < white_line < 170 or white_line > 250:
        canny = cv2.Canny(cv2.GaussianBlur(r,(3,3),0), 0, 100)
        Self_Posture =  np.argmax(canny)
    else:
        Self_Posture = 0

    if white_line > 250 and Self_Posture < 10:
        Self_Posture == len(canny)

    return Self_Posture

# 获取目标架势
def get_Target_Posture(img):
    img_roi = roi(img, x=401, x_w=553, y=29, y_h=29+1)
    b, g ,r =cv2.split(img_roi)    # 颜色通道分离

    white_line = r[0][0]
    if white_line > 190:
        canny = cv2.Canny(cv2.GaussianBlur(r,(3,3),0), 0, 100)
        Target_Posture =  np.argmax(canny)
    else:
        Target_Posture = 0

    if white_line > 250 and Target_Posture < 10:
        Target_Posture == len(canny)

    return Target_Posture
~~~

# 2 按键部分
- pysekiro
    - key_tools
        - \__init__.py
        - actions.py (动作控制)
        - direct_keys.py (控制键盘的按键)
        - get_keys.py (捕获键盘的按键)

## 2.1 捕获键盘的按键

[get_keys.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/CN/pysekiro/key_tools/get_keys.py)  
源代码： [Sentdex/pygta5/blob/master/getkeys.py](https://github.com/Sentdex/pygta5/blob/master/getkeys.py)

[keys.py](https://github.com/Sentdex/pygta5/blob/master/keys.py) 第146行至254行
~~~python
# virtual keys
vk = {
    # 'W' : 0x57,
    # 'S' : 0x53,
    # 'A' : 0x41,
    # 'D' : 0x44,
    'LSHIFT' : 0xA0,
    'SPACE'  : 0x20,

    'J' : 0x4A,
    'K' : 0x4B,

    'T' : 0x54,
    'P' : 0x50
}
~~~

检测当前正在按的键
~~~python
def key_check():
    keys = []
    for key in ['LSHIFT', 'SPACE', 'J', 'K', 'T', 'P']:    # 'W', 'S', 'A', 'D', 
        if wapi.GetAsyncKeyState(vk[key]):
            keys.append(key)
    return keys
~~~

## 2.2 控制键盘的按键

[direct_keys.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/CN/pysekiro/key_tools/direct_keys.py)  
源代码： [Sentdex/pygta5/blob/master/directkeys.py](https://github.com/Sentdex/pygta5/blob/master/directkeys.py)

PressKey 压键  
ReleaseKey 松键  

## 2.3 动作控制

[actions.py](https://github.com/ricagj/pysekiro_with_RL/blob/main/CN/pysekiro/key_tools/actions.py)   
源代码： [Sentdex/pygta5/blob/master/3.%20test_model.py](https://github.com/Sentdex/pygta5/blob/master/3.%20test_model.py)

[keys.py](https://github.com/Sentdex/pygta5/blob/master/keys.py) 第35行至143行
~~~python
# direct keys
dk = {
    'W' : 0x11,
    'S' : 0x1F,
    'A' : 0x1E,
    'D' : 0x20,
    'LSHIFT' : 0x2A,
    'SPACE'  : 0x39,

    'Y'       : 0x15,
    'NUMPAD1' : 0x4F,

    'J' : 0x24,
    'K' : 0x25,
}
~~~

e.g. 
~~~python
def Move_Forward():
    PressKey(dk['W'])      # 压键
    time.sleep(0.01)    # 按键持续时间（控制长按和短按）
    ReleaseKey(dk['W'])    # 松键
~~~

调用这个函数时，游戏就会接收到程序发送过来的按键信号，然后驱动操作对象做出相应的动作。